## doodle-bot-learn

This notebook provides a code along with the talk I gave on the use of GANs to generate images.
Here this is done using the MNIST handwritten digits dataset.

Much of this is drawn from the tensorflow models tutorials, which can be found [here](https://github.com/tensorflow/models).

## _WARNING:_

#### _Due to file size limitations, I was not able to push the tensorflow dependencies that were included in the project. The two dependencies are [slim](https://github.com/tensorflow/models/tree/master/research/slim) and [mnist](https://github.com/tensorflow/models/tree/master/research/gan/mnist). In order to run this notebook, copy those directories, and place them in this repo._

In [1]:
%matplotlib inline

In [2]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

In [3]:
os.chdir('../')  # go to main folder

!ls

CGAN.py           assets            slide_deck        visualizer.py
README.md         evaluate.py       slim              xent_score.png
__init__.py       mnist             tf_nets.py
__pycache__       notebooks         training_loss.png


In [4]:
# custom modules found in project
import visualizer
import evaluate
import tf_nets

In [5]:
# mnist examples from `tensorflow/models`.
from mnist import data_provider
from mnist import util

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# tf-slim data provider
from slim.datasets import download_and_convert_mnist

In [ ]:
import tensorflow as tf

In [ ]:
DATA_DIR = "assets/data/"
RESULT_DIR = "assets/results/"

In [ ]:
# short cuts
tfgan = tf.contrib.gan
slim = tf.contrib.slim

In [ ]:
# input pipeline
if not tf.gfile.Exists(DATA_DIR):  # check if data directory already exists
    tf.gfile.MakeDirs(DATA_DIR)

download_and_convert_mnist.run(DATA_DIR)  # download data if missing

batch_size = 32
with tf.device('/cpu:0'):  # pin it to the cpu and save gpu for propagation
    images, one_hot_labels, _ = data_provider.provide_data('train', batch_size, DATA_DIR)

In [ ]:
# check the images
imgs_to_visualize = tfgan.eval.image_reshaper(images[:20,...], num_cols=10)
visualizer.image(imgs_to_visualize, save=True)

#### Define model

In [ ]:
noise_dims = 64  # shape of noise generation

conditional_gan_model = tfgan.gan_model(generator_fn=tf_nets.generator,
                                        discriminator_fn=tf_nets.discriminator,
                                        real_data=images,
                                        generator_inputs=(tf.random_normal([batch_size, noise_dims]),one_hot_labels))

In [ ]:
# check pre-training generator images
cond_generated_data_to_visualize = tfgan.eval.image_reshaper(conditional_gan_model.generated_data[:20,...], num_cols=10)
visualizer.image(cond_generated_data_to_visualize, save=True)

In [ ]:
loss = tfgan.gan_loss(conditional_gan_model, gradient_penalty_weight=1.0)
evaluate.gan_loss(loss)  # test loss function


In [ ]:
generator_optimizer = tf.train.AdamOptimizer(0.0009, beta1=0.5)  # instantiate optimizers
discriminator_optimizer = tf.train.AdamOptimizer(0.00009, beta1=0.5)

In [ ]:
gan_train_ops = tfgan.gan_train_ops(conditional_gan_model, loss, generator_optimizer, discriminator_optimizer)

In [ ]:
images_to_eval = 500
assert images_to_eval % 10 == 0  # ensure multiples of 10

In [ ]:
random_noise = tf.random_normal([images_to_eval, 64])
one_hot_labels = tf.one_hot([i for _ in range(images_to_eval // 10) for i in range(10)], depth=10)

In [ ]:
with tf.variable_scope(conditional_gan_model.generator_scope, reuse=True):
    eval_images = conditional_gan_model.generator_fn((random_noise, one_hot_labels))

In [ ]:
reshaped_eval_imgs = tfgan.eval.image_reshaper(eval_images[:20, ...], num_cols=10)

Using a pretrained classifier to save on training time.

In [ ]:
MNIST_CLASSIFIER_FROZEN_GRAPH = 'mnist/data/classify_mnist_graph_def.pb'
xent_score = util.mnist_cross_entropy(eval_images, one_hot_labels, MNIST_CLASSIFIER_FROZEN_GRAPH)

In [ ]:
global_step = tf.train.get_or_create_global_step()
train_step_fn = tfgan.get_sequential_train_steps()
loss_values, xent_score_values = [], []

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()  # instantiate saver
    sess.run(tf.global_variables_initializer())  # run!

    with slim.queues.QueueRunners(sess):        
        start_time = time.time()  # start timer
        for i in range(3001):  # number of steps - reduced from 500 for run time
            cur_loss, _ = train_step_fn(sess, gan_train_ops, global_step, train_step_kwargs={})
            loss_values.append((i, cur_loss))

            if not i % 10:
                xent_score_values.append((i, sess.run(xent_score)))

            if not i % 100:
                print(f'Current loss: {cur_loss:.2f}')
                print(f'Current cross entropy score: {xent_score_values[-1][1]:.2f}')
                visualizer.generated_image(i, start_time, sess.run(reshaped_eval_imgs), save=True)

        #  program complete
        
        save_path = saver.save(sess, "assets/saved_models/model.ckpt")
        print(f"Model saved in file: {save_path}")

In [ ]:
# plot cross entropy scores
plt.title('Cross Entropy Score Per Step')
plt.plot(*zip(*xent_score_values))
plt.savefig('cross_entropy.png', dpi=250)

In [ ]:
plt.title('Training Loss Per Step')
plt.plot(*zip(*loss_values))
plt.savefig('loss.png', dpi=250)